In [41]:
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
from torchaudio import transforms
from torch.utils.data import Dataset
from torchmetrics.text.wer import WordErrorRate
from torchmetrics.text.cer import CharErrorRate
from torch.autograd import Variable

import pandas as pd
import os
import re
from tqdm.notebook import tqdm
import gc
from numpy.testing import assert_almost_equal
import math
import numpy as np

from conformer_model import ConformerEncoder, LSTMDecoder
from conformer_utils import *

In [42]:
DEVICE = "cuda:0"
# TRAIN_BS = 16
# TEST_BS = 10
# EPOCHS = 50
validation_transform = transforms.MelSpectrogram(sample_rate=16000, 
                                                 n_mels=80, 
                                                 hop_length=160)

In [43]:
checkpoint_path = "conformer_S_hindi_full_Mar19.pt"

char_dict = load_char_dict(checkpoint_path, DEVICE)
print(char_dict)

char_list = list(char_dict.keys())
print(char_list)

{' ': 0, 'ँ': 1, 'ं': 2, 'ः': 3, 'अ': 4, 'आ': 5, 'इ': 6, 'ई': 7, 'उ': 8, 'ऊ': 9, 'ऋ': 10, 'ए': 11, 'ऐ': 12, 'ऑ': 13, 'ओ': 14, 'औ': 15, 'क': 16, 'ख': 17, 'ग': 18, 'घ': 19, 'च': 20, 'छ': 21, 'ज': 22, 'झ': 23, 'ञ': 24, 'ट': 25, 'ठ': 26, 'ड': 27, 'ढ': 28, 'ण': 29, 'त': 30, 'थ': 31, 'द': 32, 'ध': 33, 'न': 34, 'प': 35, 'फ': 36, 'ब': 37, 'भ': 38, 'म': 39, 'य': 40, 'र': 41, 'ल': 42, 'व': 43, 'श': 44, 'ष': 45, 'स': 46, 'ह': 47, 'ा': 48, 'ि': 49, 'ी': 50, 'ु': 51, 'ू': 52, 'ृ': 53, 'े': 54, 'ै': 55, 'ॉ': 56, 'ो': 57, 'ौ': 58, '्': 59, 'ज़': 60, 'ड़': 61, 'ढ़': 62, 'फ़': 63}
[' ', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ज़', 'ड़', 'ढ़', 'फ़']


In [44]:
encoder_params = {
    "d_input": 80,
    "d_model": 144,
    "num_layers": 16,
    "conv_kernel_size": 32,
    "dropout": 0.1,
    "num_heads": 4
}

decoder_params = {
    "d_encoder": 144,
    "d_decoder": 320,
    "num_layers": 1,
    "num_classes":len(char_list)+1
}

In [45]:
encoder = ConformerEncoder(
                      d_input=encoder_params['d_input'],
                      d_model=encoder_params['d_model'],
                      num_layers=encoder_params['num_layers'],
                      conv_kernel_size=encoder_params['conv_kernel_size'], 
                      dropout=encoder_params['dropout'],
                      num_heads=encoder_params['num_heads']
                    )
  
decoder = LSTMDecoder(
                  d_encoder=decoder_params['d_encoder'], 
                  d_decoder=decoder_params['d_decoder'], 
                  num_layers=decoder_params['num_layers'],
                    num_classes= decoder_params['num_classes'])

encoder = encoder.to(DEVICE)
decoder = decoder.to(DEVICE)

In [46]:
load_model_inference(encoder, decoder, checkpoint_path, DEVICE)

(83, tensor(0.0175, device='cuda:0'), tensor(20.8077, device='cuda:0'))

In [47]:
def ConformerForward(encoder, decoder, char_decoder, spectrograms, mask, char_list, gpu=True):
    ''' Evaluate model on test dataset. '''

    encoder.eval()
    decoder.eval()

    # Move to GPU
    if gpu:
        spectrograms = spectrograms.cuda()

        mask = mask.cuda()

    with torch.no_grad():
        outputs, attention_cem = encoder(spectrograms, mask)
        outputs, decoder_cem = decoder(outputs)
 
        soft_cem = F.softmax(outputs, dim=-1)


        inds, uncollapsed_inds = char_decoder(outputs.detach())
        
        uncollapsed_predictions = []
        
        for sample1 in uncollapsed_inds:
            uncollapsed_predictions.append(int_to_text_uncollapse(sample1, len(char_list), char_list))

        collapse_predictions = []
        final_predictions = []
        for sample in inds:
            collapse_predictions.append(int_to_text_collapse(sample, len(char_list), char_list))
            final_predictions.append(int_to_text_final(sample, len(char_list), char_list))

    return attention_cem, decoder_cem, soft_cem, final_predictions, uncollapsed_predictions

In [48]:
char_decoder = GreedyCharacterDecoder().to(DEVICE)

In [49]:
def audio_process(waveform):
    mel_spec = validation_transform(waveform) # (channels, n_mels, time)
    mel_spec = mel_spec.squeeze(0).transpose(0,1)  # (time, n_mels)
    mel_spec_len = ((mel_spec.shape[0] - 1) // 2 - 1) // 2

    input_list = []
    input_list.append(mel_spec)
    input_tens = nn.utils.rnn.pad_sequence(input_list, batch_first=True)

    input_len_list = []
    input_len_list.append(mel_spec_len)

    mask = torch.ones(input_tens.shape[0], input_tens.shape[1], input_tens.shape[1])
    for i, l in enumerate(input_len_list):
        mask[i, :, :l] = 0
    return input_tens, mask.bool()

In [50]:
data_file_path = "metadata_final.tsv"
df = pd.read_csv(data_file_path, sep='\t', header=None)
df[0]="./wavs/"+df[0]

In [51]:
ref = []
pred = []
for i in tqdm(range(len(df))):
    file_path = df.iloc[i,0]
    ref.append(df.iloc[i,1])
    
    waveform, sample_rate = torchaudio.load(file_path, normalize = True)
    channel_dim = waveform.shape[0]
    if channel_dim > 1:
        waveform = torch.mean(waveform, 0, keepdim=True)
    if sample_rate!=16000:
        waveform = transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
    input_tens, mask = audio_process(waveform)

    _, _, ok, final_predictions, uncollapsed_predictions = ConformerForward(encoder,
                                                        decoder, char_decoder, input_tens, mask, 
                                                        char_list)
    pred.append(final_predictions[0])

    


  0%|          | 0/817 [00:00<?, ?it/s]

In [ ]:
# from torchaudio.models.decoder import download_pretrained_files

# files = download_pretrained_files("librispeech-4-gram")

# print(files)

In [52]:
wer = WordErrorRate()
print(wer(ref, pred))

tensor(0.2107)


In [53]:
cer = CharErrorRate()
print(cer(ref, pred))

tensor(0.0818)


Beam Search 

In [54]:
from torchaudio.models.decoder import ctc_decoder

In [71]:
LM_WEIGHT = 3.23
WORD_SCORE = -0.26

beam_search_decoder = ctc_decoder(
    lexicon="/media/rathna/New Volume/mridul/lexicon.txt",
    tokens="/media/rathna/New Volume/mridul/tokens.txt",
    lm=None,
    nbest=3,
    beam_size=1500,
    lm_weight=LM_WEIGHT,
    word_score=WORD_SCORE,
)

In [40]:
ok.shape

torch.Size([1, 317, 65])

In [74]:
beam_search_result = beam_search_decoder(ok.cpu())
beam_search_transcript = " ".join(beam_search_result[0][0].words).strip()
beam_search_wer = torchaudio.functional.edit_distance(ok, beam_search_result[0][0].words) / len(ok)

In [78]:
ref_beamsnolm = []
pred__beamsnolm = []
for i in tqdm(range(len(df))):
    file_path = df.iloc[i,0]
    ref_beamsnolm.append(df.iloc[i,1])
    
    waveform, sample_rate = torchaudio.load(file_path, normalize = True)
    channel_dim = waveform.shape[0]
    if channel_dim > 1:
        waveform = torch.mean(waveform, 0, keepdim=True)
    if sample_rate!=16000:
        waveform = transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
    input_tens, mask = audio_process(waveform)

    _, _, ok, final_predictions, uncollapsed_predictions = ConformerForward(encoder,
                                                        decoder, char_decoder, input_tens, mask, 
                                                        char_list)
    beam_search_result = beam_search_decoder(ok.cpu())
    beam_search_transcript = " ".join(beam_search_result[0][0].words).strip()
    pred__beamsnolm.append(beam_search_transcript)


  0%|          | 0/817 [00:00<?, ?it/s]

In [75]:
print(beam_search_result)

[[CTCHypothesis(tokens=tensor([ 0,  5, 35,  0, 35, 44, 51,  0, 35, 16, 59, 45, 49, 40, 57,  2,  0, 16,
        54,  0, 42, 49, 11,  0, 17, 48, 34, 48,  0, 35, 48, 34, 50,  0, 32, 54,
        34, 54,  0, 16, 48,  0,  4, 35, 34, 48,  0, 39, 48, 34, 43, 50, 40,  0,
        32, 48, 40, 49, 30, 59, 43,  0,  4, 38, 49, 34, 59, 34,  0, 37, 48, 30,
        54,  2,  0, 41, 47, 54,  2,  0, 40, 54,  0, 22, 41, 52, 41,  0, 40, 48,
        32,  0, 41, 17, 54, 18, 48,  0, 30, 37,  0, 30, 16,  0, 16, 54,  0, 42,
        49, 11,  0, 37, 47, 51, 30,  0, 37, 47, 51, 30,  0, 33, 34, 59, 40, 43,
        48, 32,  0]), words=['आप', 'पशु', 'पक्षियों', 'के', 'लिए', 'खाना', 'पानी', 'देने', 'का', 'अपना', 'मानवीय', 'दायित्व', 'अभिन्न', 'बातें', 'रहें', 'ये', 'जरूर', 'याद', 'रखेगा', 'तब', 'तक', 'के', 'लिए', 'बहुत', 'बहुत', 'धन्यवाद'], score=288.25907551044145, timesteps=tensor([  0,  10,  12,  14,  15,  18,  19,  23,  25,  27,  28,  29,  30,  31,
         32,  33,  34,  36,  37,  38,  39,  40,  41,  43,  45,  46,

In [79]:
wer = WordErrorRate()
print(wer(ref_beamsnolm, pred__beamsnolm))


tensor(0.1512)


In [84]:
cer = CharErrorRate()
print(cer(ref_beamsnolm, pred__beamsnolm))

tensor(0.0782)


Beam Search+KENLM

In [ ]:
#

In [117]:
LM_WEIGHT = 0.23
WORD_SCORE = -0.26

beam_search_decoder = ctc_decoder(
    lexicon="/media/rathna/New Volume/mridul/lexicon.txt",
    tokens="/media/rathna/New Volume/mridul/tokens.txt",
    lm="/media/rathna/New Volume/mridul/6gram.binary",
    nbest=3,
    beam_size=100,
    lm_weight=LM_WEIGHT,
    word_score=WORD_SCORE,
)

In [109]:
beam_search_result = beam_search_decoder(ok.cpu())
beam_search_transcript = " ".join(beam_search_result[0][0].words).strip()
beam_search_wer = torchaudio.functional.edit_distance(ok, beam_search_result[0][0].words) / len(ok)

In [103]:
beam_search_transcript

'आप पशु पक्षियों के लिए खाना पानी देने का अपना मानवीय दायित्व अभिन्न बात रहें ये जरुर याद रखेगा तब तक के लिए बहुत बहुत धन्यवाद'

In [118]:
ref_beamslm = []
pred__beamslm = []
for i in tqdm(range(len(df))):
    file_path = df.iloc[i,0]
    ref_beamslm.append(df.iloc[i,1])
    
    waveform, sample_rate = torchaudio.load(file_path, normalize = True)
    channel_dim = waveform.shape[0]
    if channel_dim > 1:
        waveform = torch.mean(waveform, 0, keepdim=True)
    if sample_rate!=16000:
        waveform = transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
    input_tens, mask = audio_process(waveform)

    _, _, ok, final_predictions, uncollapsed_predictions = ConformerForward(encoder,
                                                        decoder, char_decoder, input_tens, mask, 
                                                        char_list)
    beam_search_result = beam_search_decoder(ok.cpu())
    beam_search_transcript = " ".join(beam_search_result[0][0].words).strip()
    pred__beamslm.append(beam_search_transcript)

  0%|          | 0/817 [00:00<?, ?it/s]

In [119]:
wer = WordErrorRate()
print(wer(ref_beamslm, pred__beamslm))

tensor(0.0950)


In [120]:
cer = CharErrorRate()
print(cer(ref_beamslm, pred__beamslm))

tensor(0.0643)


In [121]:
for i in range(len(ref)):
    word_error_rate = wer(ref_beamslm[i], pred__beamslm[i])
    if word_error_rate == 0:
        continue
    else:
        print("REFERENCE")
        print(ref_beamslm[i])
        print("PRED")
        print(pred__beamslm[i])

REFERENCE
नए विषयो के साथ नए प्रेरक उदाहरणों के साथ नए नए संदेशो को समेटे हुए
PRED
नए विषयों के साथ नए प्रेरक उदाहरणों के साथ नए नए संदेशो को समेटे हुए
REFERENCE
और संदेश किस विषय को लेकर मिली है ये विषय ऐसा है जो इतिहास वर्तमान और भविष्य
PRED
और संदेश की इस विषय को लेकर मिली है ये विषय ऐसा है कि इतिहास वर्तमान और भविष्य
REFERENCE
इस चौदह अप्रैल को बाबा साहेब अम्बेडकर की जन्म जयन्ती पर प्रधानमंत्री संग्रहालय का लोकार्पण हुआ है
PRED
इस चौदह अप्रैल को बाबा साहेब अम्बेडकर की जन्मजयंती पर प्रधानमंत्री संग्रहालय का लोकार्पण हुआ है
REFERENCE
इसेदेश के नागरिकों के लिए खोल दिया गया है एक श्रोता हैं श्रीमान सार्थक जीसार्थक जी गुरुग्राम में रहते हैं
PRED
इसेदेश के नागरिकों के लिए खोल दिया गया है एक श्रोता हैं श्रीमान सार्थक जी सार्थक जी गुरुग्राम में रहते हैं
REFERENCE
उन्हें महसूस हुआ कि वे अपने देश और देश का नेतृत्व करने वालों के बारे में काफी कुछ जानते ही नहीं हैं उन्होंने
PRED
उन्हें महसूस हुआ कि वे अपने देश और देश का नेतृत्व करने वालों के बारे में काफी कुछ जानते ही नहीं है उन्होंने
REFERENC

In [91]:
for i in range(len(ref)):
    word_error_rate = wer(ref_beamslm[i], pred__beamslm[i])
    if word_error_rate == 0:
        continue
    else:
        print("REFERENCE")
        print(ref_beamslm[i])
        print("PRED")
        print(pred__beamslm[i])

REFERENCE
मेरे प्यारे देशवासियो नमस्कार
PRED

REFERENCE
नए विषयो के साथ नए प्रेरक उदाहरणों के साथ नए नए संदेशो को समेटे हुए
PRED
उदाहरण है
REFERENCE
एक बार फिर मैं आपसे मन की बात करने आया हूँ जानते हैं इस बार मुझे सबसे ज्यादा चिट्ठियाँ
PRED

REFERENCE
और संदेश किस विषय को लेकर मिली है ये विषय ऐसा है जो इतिहास वर्तमान और भविष्य
PRED

REFERENCE
तीनों से जुड़ा हुआ है मैं बात कर रहा हूँ देश को मिले नए प्रधानमंत्री संग्रहालय की
PRED
प्रधानमंत्री संग्रहालय
REFERENCE
इस चौदह अप्रैल को बाबा साहेब अम्बेडकर की जन्म जयन्ती पर प्रधानमंत्री संग्रहालय का लोकार्पण हुआ है
PRED
प्रधानमंत्री संग्रहालय की
REFERENCE
इसेदेश के नागरिकों के लिए खोल दिया गया है एक श्रोता हैं श्रीमान सार्थक जीसार्थक जी गुरुग्राम में रहते हैं
PRED
सार्थक जी ने
REFERENCE
और पहला मौका मिलते ही वो प्रधानमंत्री संग्रहालय देख आए हैं सार्थक जी ने नमो एप पर जो संदेश मुझे लिखा है
PRED
प्रधानमंत्री संग्रहालय की
REFERENCE
वो बहुत इन्टरिस्टिंग है उन्होंने लिखा है कि वो बरसों से न्यूज़ चैनल देखते हैं अखबार पढ़ते हैं सोशल मीडिया से भी कनेक्टेड

In [68]:
words=set()
temp_charset = set()
for sentence in train_df[1]:
    sentence=sentence.split(" ")
#     print(sentence)
    for word in sentence:
        words.add(word)
        for char in word:
            temp_charset.add(char)
        
words = sorted(list(words))
# print(words)
with open('lexicon.txt', 'w') as f:
    for word in words:
        f.write(word+" ")
        word_split = ' '.join(list(word))
        print(word_split)
        f.write(word_split+" |")
        f.write('\n')

अ ं ग
अ ं ग ् र े ज ो ं
अ ं ज ी र
अ ं त र र ा ष ् ट ् र ी य
अ ं त र ा ल
अ ं त र ् ग त
अ ं त र ् र ा ष ् ट ् र ी य
अ ं द र
अ क ् त ू ब र
अ क ् ष य
अ क ् स र
अ ख ब ा र
अ ग र
अ ग ल ा
अ ग ल ी
अ ग ल े
अ ग स ् त
अ ग ् र ि म
अ च ा र
अ च ् छ ा
अ च ् छ ी
अ च ् छ े
अ ज म े र
अ ट ल
अ ट ू ट
अ ट ् ठ ा ई स
अ ट ् ठ ा र ह
अ ट ् र ै क ् ट ि व
अ ठ ा र ह
अ त ि थ ि
अ त ी त
अ त ् य ं त
अ थ क
अ द ् भ ु त
अ ध ि क
अ ध ि क त र
अ ध ि क ा र
अ ध ू र ी
अ ध ् य क ् ष त ा
अ न ं त
अ न ग ि न त
अ न ज ा न
अ न म ो ल
अ न ल ा इ न
अ न व र त
अ न स ु न े
अ न ा ज
अ न ि ल
अ न ु भ व
अ न ु भ व ो ं
अ न ु भ ू त ि
अ न ु र ू प
अ न ु स र ण
अ न ू ठ ा
अ न ू ठ े
अ न े क
अ न ो ख ा
अ न ् ड र
अ न ् त र ् र ा ष ् ट ् र ी य
अ न ् न
अ न ् य
अ प
अ प न ा
अ प न ा ए ं
अ प न ा ए ं ग े
अ प न ा न ा
अ प न ा न े
अ प न ी
अ प न े
अ प न ो ं
अ प ल ो ड
अ प े क ् ष ा
अ प ् र ै ल
अ प ् ल ा इ
अ प ् स
अ फ स र ो ं
अ ब
अ भ ि
अ भ ि न व
अ भ ि न ् न
अ भ ि भ ा व क ो ं
अ भ ि य ा न
अ भ ि व ् य क ् त
अ भ ी
अ भ ू त प ू र ् व
अ भ ् य ा स
अ म र
अ म र न ा थ
अ म ा र त म
अ म 

In [56]:
print(char_list)

[' ', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ज़', 'ड़', 'ढ़', 'फ़']


In [48]:
print(temp_charset)

{'r', 'e', 'y', 'h', 'a', 'f', 'n', 'z', 'l', 'w', 'x', 'u', 'p', 'v', 'm', 'd', 'i', 'b', 'g', 'o', 'k', 'j', 't', 'c', 'q', 's'}


In [69]:
with open('tokens.txt', 'w') as f:
    for char in char_list:
        f.write(char)
        f.write("\n")

In [92]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\।\’]'

train_df = pd.read_csv('metadata_final.tsv', sep='\t', header=None)
# valid_df = pd.read_csv('../input/dlsprint/validation.csv')

In [93]:
with open('text.txt', 'w') as f:
    for sentence in train_df[1]:
        f.write(re.sub(chars_to_ignore_regex, '', sentence))
        f.write('\n')

In [94]:
! /home/rathna/kenlm/build/bin/lmplz -o 6 <"text.txt" > "6gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /media/rathna/New Volume/mridul/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 18779 types 3485
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:41820 2:812777088 3:1523957120 4:2438331392 5:3555899904 6:4876662784
Statistics:
1 3485 D1=0.648855 D2=1.13251 D3+=1.43853
2 13029 D1=0.821794 D2=1.26225 D3+=1.48992
3 17147 D1=0.937604 D2=1.43332 D3+=1.7385
4 17549 D1=0.981668 D2=1.60006 D3+=2.10757
5 17024 D1=0.99414 D2=1.70176 D3+=0.614065
6 16285 D1=0.995343 D2=1.76426 D3+=3
Memory estimate for binary LM:
type      kB
probing 1892 assuming -p 1.5
probing 2285 assuming -r models -p 1.5
trie     876 without quantization
trie     468 assuming -q 8 -b 8 quantization 
trie     815 assuming -a 22 array pointer compression
trie     407 assuming -a 22 -q 

In [95]:
! /home/rathna/kenlm/build/bin/build_binary 6gram.arpa 6gram.binary

Reading 6gram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS
